In [ ]:
import os
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
import numpy as np
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical



In [ ]:
def preprocess_data_CNN(audio_dir, spectrogram_dir):
  file_names = []
  for audio_subdir in os.walk(audio_dir):
      file_names += [join(audio_subdir[0], f) for f in listdir(audio_subdir[0]) if isfile(join(audio_subdir[0], f)) and '.wav' in f]
  for file_name in file_names:
    print(file_name)
    audio_path =  file_name
    spectogram_path = spectrogram_dir + file_name.split("/")[2].replace('.wav', '.png')
    wav_to_spectrogram(audio_path, spectogram_path)
    

In [ ]:
def wav_to_spectrogram(audio_path, save_path):
  spectrogram_dimensions=(64, 64)
  sample_rate, samples = wav.read(audio_path)
  fig = plt.figure()
  fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
  ax = plt.Axes(fig, [0., 0., 1., 1.])
  ax.set_axis_off()
  fig.add_axes(ax)
  ax.specgram(samples, cmap='gray_r', Fs=2, noverlap=16)
  ax.xaxis.set_major_locator(plt.NullLocator())
  ax.yaxis.set_major_locator(plt.NullLocator())
  fig.savefig(save_path, bbox_inches="tight", pad_inches=0)

In [ ]:
audio_folder = "data/"
spectrogram_folder = "spectrograms/"
os.makedirs(spectrogram_folder, exist_ok=True)
preprocess_data_CNN(audio_folder, spectrogram_folder)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import tensorflow as tf

imagesDir = "spectrograms/"
trainset = []
testset = []
for file in os.listdir(imagesDir):
  if file != ".ipynb_checkpoints":
    label = file.split('_')[0]
    sample_number = file.split('_')[2]
    img = tf.keras.utils.load_img(imagesDir+file)
#     print(img)
    if sample_number in ['1.','3.','5.','7.','9.']:
      testset.append([tf.keras.utils.img_to_array(img), label])
    else:
      trainset.append([tf.keras.utils.img_to_array(img), label])

    
X_train = [item[0] for item in trainset]
Y_train = [item[1] for item in trainset]
X_test = [item[0] for item in testset]
Y_test = [item[1] for item in testset]

X_train = np.asanyarray(X_train)
X_test = np.asanyarray(X_test)
X_train /= 255
X_test /= 255

Y_train = np.asanyarray(Y_train)
Y_train = to_categorical(Y_train)
Y_test = np.asanyarray(Y_test)
Y_test = to_categorical(Y_test)





In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras import models

data_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
def basic_cnn():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=data_shape))
  model.add(BatchNormalization())
  model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
  model.add(BatchNormalization())
  model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Dense(64, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy'])
  return model
  

In [ ]:
model_CNN = basic_cnn()
history = model_CNN.fit(X_train, Y_train, batch_size = 50, validation_split=0.2, epochs = 50, verbose = 1)
model_CNN.evaluate(X_test,Y_test)
model_CNN.save("model_CNN.h5")



In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["Train","Validation"], loc="upper left")
# plt.savefig("acc.png")
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["Train","Validation"], loc="upper left")
plt.show()
# plt.savefig("los.png")
